# CDK2: Model Selection - y values shuffled
### Validation Method

In [ ]:
import pandas as pd
import numpy as np
import glob, sys, os
sys.path.append('..')

In [ ]:
from modules.plotting_metrics import PlotMetric
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='white', context='talk', font_scale=0.8)

In [ ]:
file_name = './df_DkSc_results_COCRYS_DEKOIS_DUD.pkl'
X_merged_dksc = pd.read_pickle(file_name)
# Extract activity column
y_true_merged = X_merged_dksc['activity']
# Drop column from merged_dkksc
X_merged_dksc = X_merged_dksc.drop('activity', axis=1)
X_merged_dksc.shape
y_true_merged.loc['DEKOIS'].sum()

## Scaffold Splitting

In [ ]:
#*************************************************
# Functions to compute stratify scaffold splitting
#*************************************************
sys.path.append('../2_Docking_analysis/')
from scaffold_splitter import train_test_scaffold_split

In [ ]:
# Compute or load the dataframe containing the Generic Murcko Scaffolds
file = '../2_Docking_analysis/df_COCRYS_DUD_DEKOIS_Murcko_Scaffolds_SMILES.obj'

df_scff_murcko = pd.read_pickle(file)
df_scff_murcko.shape

In [ ]:
%run 4_Helper_Functions_Model_Selection_Grid_Search.ipynb

#  Hyperparameter Tunning: Grid Search

In [ ]:
def randomize_y_labels(y_target, random_chi=0.1):
    '''Función para distribuir de forma aleatoria una fracción 
    chi del vector de etiquetas, de forma estratificada'''
    
    # Make a copy of the original vector
    y_copy = y.copy()
    
    # Get the number of actives inside the y_target vector
    n_actives = y_target.sum()
    random_size = np.floor(random_chi * n_actives)
    # Initialize the counters
    act_count = random_size
    inact_count = random_size
    
    # Create the randomized list of idexes
    idx_shuffled = np.random.choice(range(len(y)), len(y), replace=False)
    # iterate over idx_shuffled until act and inact counters == 0
    for l in idx_shuffled:
        if act_count > 0:
            if y_copy[l] == 1: # Is active, then change it to inactive
                y_copy[l] = 0
                act_count = act_count - 1
                continue
            if inact_count > 0: # If is inactive, change it to active
                y_copy[l] = 1
                inact_count = inact_count - 1
                continue
        else:
            break
    return(y_copy)
    

***
<h2 style='background-color: #F9E5AB; padding: 5px;'>
    Merged libraries: Shuffle *y* target values in the train set
</h2>
<div style='background-color: #FE8460; min-height: 5px'></div>

#### DEKOIS, DUD and COCRYS are  treated as one unique library
#### Target values (y) will be shuffled

In [ ]:
# Train and test over 
X = X_merged_dksc
# ***** Permutate y values *****
y = y_true_merged#.sample(frac=1, replace=False)

library = 'Merged'
scaffold_series = df_scff_murcko['scff_generic']

# Create an empty dictionary to save results

<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Logistic Regression</h3>
<b>Merged Libraries</b>

In [ ]:
%%time
from sklearn.linear_model import LogisticRegression

chi_values = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 1]

for chi in chi_values:
    print('valor chi:', chi)
    for rep in range(10):
        print('rep', rep)
        y_rd = randomize_y_labels(y, chi)

        estimator_name = 'LogReg'
        estimator_name = estimator_name + '_chi' + str(chi) + '_rep' + str(rep)
        hyperparams = {'C': 1, 
                       'penalty': 'l1', 
                       'solver': 'liblinear', 
                       'max_iter': 150} 
        estimator = LogisticRegression(**hyperparams)
        
        # RANDOM Train test splitting
        split_and_gs(library, library, estimator_name,
                     X, y_rd, estimator, {},
                     splitting='random', test_size=0.25, 
                     scaffold_series=None, verbose=False)

In [ ]:
%%time
from sklearn.linear_model import LogisticRegression

chi_values = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 1]

for chi in chi_values:
    print('valor chi:', chi)
    for rep in range(10):
        print('rep', rep)
        y_rd = randomize_y_labels(y, chi)

        estimator_name = 'LogReg'
        estimator_name = estimator_name + '_chi' + str(chi) + '_rep' + str(rep)
        hyperparams = {'C': 1.0, 
                       'penalty': 'l1', 
                       'solver': 'liblinear', 
                       'max_iter': 150}
        estimator = LogisticRegression(**hyperparams)

        # SCAFFOLD Train test splitting
        split_and_gs(library, library, estimator_name,
                     X, y_rd, estimator, {},
                     splitting='scaffold', test_size=0.25, 
                     scaffold_series=scaffold_series, verbose=False)

<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: X Gradient Boosting </h3>
<b>Merged Libraries</b>

In [ ]:
%%time

from xgboost import XGBClassifier

chi_values = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 1]

for chi in chi_values:
    print('valor chi:', chi)
    for rep in range(10):
        print('rep', rep)
        y_rd = randomize_y_labels(y, chi)

        estimator_name = 'XGB_tree'
        estimator_name = estimator_name + '_chi' + str(chi) + '_rep' + str(rep)
        estimator = XGBClassifier()
        hyperparams = {'subsample': 0.5, 
                       'n_estimators': 200, 'max_depth': 10,
                       'learning_rate': 0.1, 'gamma': 1, 
                       'colsample_bytree': 0.5, 'alpha': 0.5}

        # RANDOM Train test splitting
        split_and_gs(library, library, estimator_name,
                     X, y_rd, estimator, {},
                     splitting='random', test_size=0.25, 
                     scaffold_series=None, verbose=False)

In [ ]:
%%time

from xgboost import XGBClassifier

chi_values = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 1]

for chi in chi_values:
    print('valor chi:', chi)
    for rep in range(10):
        print('rep', rep)
        y_rd = randomize_y_labels(y, chi)

        estimator_name = 'XGB_tree'
        estimator_name = estimator_name + '_chi' + str(chi) + '_rep' + str(rep)
        estimator = XGBClassifier()
        hyperparams = {'subsample': 0.5, 
                       'n_estimators': 200, 'max_depth': 10,
                       'learning_rate': 0.1, 'gamma': 1, 
                       'colsample_bytree': 0.5, 'alpha': 0.5}

        # RANDOM Train test splitting
        split_and_gs(library, library, estimator_name,
                     X, y_rd, estimator, {},
                     splitting='scaffold', test_size=0.25, 
                     scaffold_series=scaffold_series, verbose=False) 

<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Radial Basis Function SVM</h3>
<b>Merged Libraries</b>

In [ ]:
%%time
from sklearn.svm import SVC
chi_values = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 1]

for chi in chi_values:
    print('valor chi:', chi)
    for rep in range(10):
        print('rep', rep)
        y_rd = randomize_y_labels(y, chi)
    
        estimator_name = 'rbfSVC'
        estimator_name = estimator_name + '_chi' + str(chi)
        estimator = SVC(kernel = 'rbf', probability=True)
        hyperparams = {'kernel': 'rbf', 
                       'probability': True, 
                       'C': 10.0, 'gamma': 0.01} 

        # RANDOM Train test splitting
        split_and_gs(library, library, estimator_name,
                     X, y_rd, estimator, {},
                     splitting='random', test_size=0.25, 
                     scaffold_series=None, verbose=False)

In [ ]:
%%time
from sklearn.svm import SVC
chi_values = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 1]

for chi in chi_values:
    print('valor chi:', chi)
    for rep in range(10):
        print('rep', rep)
        y_rd = randomize_y_labels(y, chi)
    
        estimator_name = 'rbfSVC'
        estimator_name = estimator_name + '_chi' + str(chi)
        estimator = SVC(kernel = 'rbf', probability=True)
        hyperparams = {'kernel': 'rbf', 
                       'probability': True, 
                       'C': 10.0, 'gamma': 0.01} 
    
        # SCAFFOLD Train test splitting
        split_and_gs(library, library, estimator_name,
                     X, y_rd, estimator, hyperparams,
                     splitting='scaffold', test_size=0.25, 
                     scaffold_series=scaffold_series, verbose=False)

***
<h2 style='background-color: #F9E5AB; padding: 5px;'>
    DEKOIS: Shuffle *y* target values in the train set
</h2>
<div style='background-color: #FE8460; min-height: 5px'></div>

#### DEKOIS 
#### Target values (y) will be shuffled

In [ ]:
library = 'DEKOIS'

# Train and test over DUDU
X = X_merged_dksc.loc[library]
y = y_true_merged.loc[library]
scaffold_series = df_scff_murcko['scff_generic'].loc[library]

<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Logistic Regression</h3>
<b>Merged Libraries</b>

In [ ]:
%%time
from sklearn.linear_model import LogisticRegression

chi_values = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 1]

for chi in chi_values:
    print('valor chi:', chi)
    for rep in range(10):
        print('rep', rep)
        y_rd = randomize_y_labels(y, chi)

        estimator_name = 'LogReg'
        estimator_name = estimator_name + '_chi' + str(chi) + '_rep' + str(rep)
        hyperparams = {'C': 1, 
                       'penalty': 'l1', 
                       'solver': 'liblinear', 
                       'max_iter': 150} 
        estimator = LogisticRegression(**hyperparams)
        
        # RANDOM Train test splitting
        split_and_gs(library, library, estimator_name,
                     X, y_rd, estimator, {},
                     splitting='random', test_size=0.25, 
                     scaffold_series=None, verbose=False)

In [ ]:
%%time
from sklearn.linear_model import LogisticRegression

chi_values = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 1]

for chi in chi_values:
    print('valor chi:', chi)
    for rep in range(10):
        print('rep', rep)
        y_rd = randomize_y_labels(y, chi)

        estimator_name = 'LogReg'
        estimator_name = estimator_name + '_chi' + str(chi) + '_rep' + str(rep)
        hyperparams = {'C': 1.0, 
                       'penalty': 'l1', 
                       'solver': 'liblinear', 
                       'max_iter': 150}
        estimator = LogisticRegression(**hyperparams)

        # SCAFFOLD Train test splitting
        split_and_gs(library, library, estimator_name,
                     X, y_rd, estimator, {},
                     splitting='scaffold', test_size=0.25, 
                     scaffold_series=scaffold_series, verbose=False)

***
<h2 style='background-color: #F9E5AB; padding: 5px;'>
    DUD: Shuffle *y* target values in the train set
</h2>
<div style='background-color: #FE8460; min-height: 5px'></div>

#### DUD 
#### Target values (y) will be shuffled

In [ ]:
library = 'DUD'

# Train and test over DUDU
X = X_merged_dksc.loc[library]
y = y_true_merged.loc[library]
scaffold_series = df_scff_murcko['scff_generic'].loc[library]

<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Logistic Regression</h3>
<b>Merged Libraries</b>

In [ ]:
%%time
from sklearn.linear_model import LogisticRegression

chi_values = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 1]

for chi in chi_values:
    print('valor chi:', chi)
    for rep in range(10):
        print('rep', rep)
        y_rd = randomize_y_labels(y, chi)

        estimator_name = 'LogReg'
        estimator_name = estimator_name + '_chi' + str(chi) + '_rep' + str(rep)
        hyperparams = {'C': 1, 
                       'penalty': 'l1', 
                       'solver': 'liblinear', 
                       'max_iter': 150} 
        estimator = LogisticRegression(**hyperparams)
        
        # RANDOM Train test splitting
        split_and_gs(library, library, estimator_name,
                     X, y_rd, estimator, {},
                     splitting='random', test_size=0.25, 
                     scaffold_series=None, verbose=False)

In [ ]:
%%time
from sklearn.linear_model import LogisticRegression

chi_values = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 1]

for chi in chi_values:
    print('valor chi:', chi)
    for rep in range(10):
        print('rep', rep)
        y_rd = randomize_y_labels(y, chi)

        estimator_name = 'LogReg'
        estimator_name = estimator_name + '_chi' + str(chi) + '_rep' + str(rep)
        hyperparams = {'C': 1.0, 
                       'penalty': 'l1', 
                       'solver': 'liblinear', 
                       'max_iter': 150}
        estimator = LogisticRegression(**hyperparams)

        # SCAFFOLD Train test splitting
        split_and_gs(library, library, estimator_name,
                     X, y_rd, estimator, {},
                     splitting='scaffold', test_size=0.25, 
                     scaffold_series=scaffold_series, verbose=False)

# Results

In [ ]:

row_names = ['Train_set', 'Test_set', 'Model name', 'Split', 
             'N_actives_train', 'N_actives_test', 'N_mols_train', 'Num_mols_test',
            'Mean-CV-ROC', 'ROC-AUC_train', 'ROC-AUC_test', 'best_params',
            'DkS_max_ROC_train',  'DkSc_med_ROC_train', 'DkSc_mean_ROC_train', 
            'DkS_max_ROC_test',  'DkSc_med_ROC_test', 'DkSc_mean_ROC_test'
            ]


y_shuffled_ml_model_selecion = pd.DataFrame(results_dict, index = row_names).T
y_shuffled_ml_model_selecion.to_pickle('df_y_shuffled_ml_model_selecion.pkl')

In [ ]:
# y_shuffled_ml_model_selecion

## Analysis

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(context='talk', style=None, font_scale=0.8)
import warnings
warnings.filterwarnings("ignore")
y_shuffled_ml_model_selecion = pd.read_pickle('./df_y_shuffled_ml_model_selecion.pkl')

# Create a column to indicate the chi value
df = y_shuffled_ml_model_selecion
df['chi'] =  df['Model name'].str.split('_chi').str[1]
df['Model name'] = df['Model name'].str.split('_').str[0]
df = df.reset_index(drop=True)
df.set_index(['Train_set', 'Test_set', 'Model name', 'Split'], inplace=True)
df = df.sort_index(level=1, axis=1)

In [ ]:
def plot_results(db, split):
    for ml_clf in df.index.get_level_values(2).unique():
        idx = (db, db, ml_clf, split)
        roc_cols = ['Mean-CV-ROC', 'ROC-AUC_train', 'ROC-AUC_test']
        try:
            df.loc[idx].plot(x='chi', 
                             y=roc_cols, 
                             figsize=(10,6),
                             ylim=(0.2, 1.01), 
                             title=db + '-' + split + '-' + ml_clf)
            plt.xlabel('Randomized fraction')
            plt.ylabel('ROC-AUC')
            plt.grid(True)
            plt.axhline(y=0.5, color='black', linestyle=':')
        except: None

In [ ]:
plot_results('Merged', 'random')

In [ ]:
plot_results('Merged', 'scaffold')

In [ ]:
plot_results('DUD', 'scaffold')

In [ ]:
plot_results('DEKOIS', 'scaffold')